In [1]:
#dataset
import json
import pandas as pd
import re
import jieba 
import jieba.analyse
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from datetime import datetime
from imblearn.over_sampling import RandomOverSampler
import scipy.stats as stats

In [3]:
#data selection
df_final_balanced_all= pd.read_csv('df_merged_balanced_all_columns.csv')

In [11]:
df_final_balanced_all.head(2)

,ID,label,nickname,gender,profile,birthday,num_of_follower,num_of_following,all_tweet_count,original_tweet_count,repost_tweet_count,tweets
0,15315,0,南琅子柒,女,很会害羞,水瓶座,413,1672,3634,50,32,"[{'tweet_content': '冲', 'posting_time': '2020-..."
1,17978,0,en嗯en_怪好,女,我想去一个地方、和你或者是自己看风吹麦浪,无,488,145,1969,41,57,[{'tweet_content': '拥抱是这个世界上最暖心的动作吧 生活中难免遇到挫折 ...


In [4]:
df_final_balanced_all2=df_final_balanced_all.copy()

In [6]:
df_final_balanced_all2.head(2)

,ID,label,nickname,gender,profile,birthday,num_of_follower,num_of_following,all_tweet_count,original_tweet_count,repost_tweet_count,tweets
0,15315,0,南琅子柒,女,很会害羞,水瓶座,413,1672,3634,50,32,"[{'tweet_content': '冲', 'posting_time': '2020-..."
1,17978,0,en嗯en_怪好,女,我想去一个地方、和你或者是自己看风吹麦浪,无,488,145,1969,41,57,[{'tweet_content': '拥抱是这个世界上最暖心的动作吧 生活中难免遇到挫折 ...


In [7]:
df_final_balanced_all_tweets = df_final_balanced_all2[['ID', 'label']].copy()

In [12]:
#test
import ast

# Assuming 'tweets' column contains string representations of lists
df_final_balanced_all2['tweets'] = df_final_balanced_all2['tweets'].apply(ast.literal_eval)

# Now apply your lambda function
df_final_balanced_all_tweets['tweet_content'] = df_final_balanced_all2['tweets'].apply(lambda x: [d['tweet_content'] for d in x])


In [13]:
df_final_balanced_all_tweets.head()

,ID,label,tweet_content
0,15315,0,"[冲, 害 我真的挺吃lyw的颜 但一想到他只有14岁我就, 足以表达我减肥的决心了, 无,..."
1,17978,0,[拥抱是这个世界上最暖心的动作吧 生活中难免遇到挫折 坎坷 不安 在一个人慌乱的时候 也许有...
2,221,0,"[下雨天别穿max97 会滑倒 坐水里 亲测, 我艾特曾子赟相信大家都不会有意见的对吧, 麻..."
3,16604,0,"[有一种幸福～叫做一桌菜然后看着家人朋友满足的吃完 就是没人帮忙刷碗, 老规矩, 昨晚本想做..."
4,20755,0,"[无, 无, 无, 无, 无, 无, 无, 无, 无, 无, 无, 无, 无, 无, 无, ..."


In [14]:
# after data balance1
grouped_label = df_final_balanced_all_tweets.groupby(['label', ]).size().reset_index(name='count')

In [15]:
grouped_label

,label,count
0,0,10325
1,1,10325


In [16]:
# Save final data to CSV file
df_final_balanced_all_tweets.to_csv('df_final_balanced_ID_label_tweets.csv', index=False)

In [17]:
#data clean
balanced_df_all = pd.read_csv('df_final_balanced_ID_label_tweets.csv')

In [19]:
balanced_df_all.loc[balanced_df_all['ID'] == 15315]

,ID,label,tweet_content
0,15315,0,"['冲', '害 我真的挺吃lyw的颜 但一想到他只有14岁我就', '足以表达我减肥的决心..."


In [20]:
balanced_df_all.loc[balanced_df_all['ID'] == 221]

,ID,label,tweet_content
2,221,0,"['下雨天别穿max97 会滑倒 坐水里 亲测', '我艾特曾子赟相信大家都不会有意见的对吧..."
10545,221,1,['您居然向我这样的反英灵施于如此恩情如梦般喜悦如朝露般缥缈―――因为这样一来分别时会很痛苦...


In [22]:
# Load stopwords
with open('cn_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

In [24]:
def clean_text(text):
   
    # Remove numbers, emojis, punctuations,special charts,shared posts, hashtags
    # Only Chinese characters are reserved.
    text = re.sub(r'[^\u4e00-\u9fff]+', '', text)
    
    # Remove weibo stop words
    for word in stopwords:
        if word in text:
            text = re.sub(word, '', text)   
    return text

In [25]:
balanced_df_all_cleaned = balanced_df_all.copy()

In [26]:
balanced_df_all_cleaned.loc[:, 'tweet_content'] = balanced_df_all_cleaned['tweet_content'].apply(clean_text)

In [27]:
balanced_df_all_cleaned

,ID,label,tweet_content
0,15315,0,害真挺吃颜想岁足表达减肥决心姐妹谈恋爱什感觉全世界闺蜜介绍成功行淦近什新情侣装评手评寻思俩关...
1,17978,0,拥抱世界暖心动作生活中难免遇挫折坎坷安慌乱许时拥抱会充满量继续世界抗衡愿生山树栖心爱春赏花夏...
2,221,0,雨天穿会滑倒坐水里亲测艾特子赟相信家会意见麻雀妈妈问麻雀天扎什头发麻雀说啾啾晚子赟帮辫头发突...
3,16604,0,种幸福做桌菜然家朋友满足吃完没帮忙刷碗老规矩昨晚想做早餐奶包发酵忘遛弯回变成馒头样子朋友吃午...
4,20755,0,日清明青草疫情中牺牲医护员公安干警基层干部线工作逝世胞表示沉痛悼念情病毒面前选择逆行选择死生...
...,...,...,...
20645,10321,1,意做件错事真久久原谅近发生事情绪波动房间突然爆哭然雨天跑出外面冻两时爸妈姨表哥电话接妈妈外面...
20646,10322,1,太喜欢张教授说话特实危言耸听然整体说认识医生亲戚朋友医生医生子女微博关注医生部分说话客观尤专...
20647,10323,1,种毒药断送爱常常听说爱情蜜糖深陷中会沉醉岂知份爱情美会意想原理结束常常意想称爱情毒药爱情会什...
20648,10324,1,果原谅痛苦值降低现贼悔吃麻辣串洗完澡新换衣服全味难受天新闻男熬夜猝死死女抑郁症确诊天昨天加药...


In [28]:
# save to csv
balanced_df_all_cleaned.to_csv('balanced_df_all_cleaned.csv', index=False, encoding='utf-8')

In [31]:
#tokenication
balanced_df_all_cleaned = pd.read_csv('balanced_df_all_cleaned.csv')

In [32]:
balanced_df_all_cleaned.loc[balanced_df_all_cleaned['ID'] == 10321]

,ID,label,tweet_content
20645,10321,1,意做件错事真久久原谅近发生事情绪波动房间突然爆哭然雨天跑出外面冻两时爸妈姨表哥电话接妈妈外面...


In [33]:
balanced_df_all_cleaned

,ID,label,tweet_content
0,15315,0,害真挺吃颜想岁足表达减肥决心姐妹谈恋爱什感觉全世界闺蜜介绍成功行淦近什新情侣装评手评寻思俩关...
1,17978,0,拥抱世界暖心动作生活中难免遇挫折坎坷安慌乱许时拥抱会充满量继续世界抗衡愿生山树栖心爱春赏花夏...
2,221,0,雨天穿会滑倒坐水里亲测艾特子赟相信家会意见麻雀妈妈问麻雀天扎什头发麻雀说啾啾晚子赟帮辫头发突...
3,16604,0,种幸福做桌菜然家朋友满足吃完没帮忙刷碗老规矩昨晚想做早餐奶包发酵忘遛弯回变成馒头样子朋友吃午...
4,20755,0,日清明青草疫情中牺牲医护员公安干警基层干部线工作逝世胞表示沉痛悼念情病毒面前选择逆行选择死生...
...,...,...,...
20645,10321,1,意做件错事真久久原谅近发生事情绪波动房间突然爆哭然雨天跑出外面冻两时爸妈姨表哥电话接妈妈外面...
20646,10322,1,太喜欢张教授说话特实危言耸听然整体说认识医生亲戚朋友医生医生子女微博关注医生部分说话客观尤专...
20647,10323,1,种毒药断送爱常常听说爱情蜜糖深陷中会沉醉岂知份爱情美会意想原理结束常常意想称爱情毒药爱情会什...
20648,10324,1,果原谅痛苦值降低现贼悔吃麻辣串洗完澡新换衣服全味难受天新闻男熬夜猝死死女抑郁症确诊天昨天加药...


In [34]:
balanced_df_all_cleaned2 = balanced_df_all_cleaned.copy()

In [35]:
balanced_df_all_cleaned2

,ID,label,tweet_content
0,15315,0,害真挺吃颜想岁足表达减肥决心姐妹谈恋爱什感觉全世界闺蜜介绍成功行淦近什新情侣装评手评寻思俩关...
1,17978,0,拥抱世界暖心动作生活中难免遇挫折坎坷安慌乱许时拥抱会充满量继续世界抗衡愿生山树栖心爱春赏花夏...
2,221,0,雨天穿会滑倒坐水里亲测艾特子赟相信家会意见麻雀妈妈问麻雀天扎什头发麻雀说啾啾晚子赟帮辫头发突...
3,16604,0,种幸福做桌菜然家朋友满足吃完没帮忙刷碗老规矩昨晚想做早餐奶包发酵忘遛弯回变成馒头样子朋友吃午...
4,20755,0,日清明青草疫情中牺牲医护员公安干警基层干部线工作逝世胞表示沉痛悼念情病毒面前选择逆行选择死生...
...,...,...,...
20645,10321,1,意做件错事真久久原谅近发生事情绪波动房间突然爆哭然雨天跑出外面冻两时爸妈姨表哥电话接妈妈外面...
20646,10322,1,太喜欢张教授说话特实危言耸听然整体说认识医生亲戚朋友医生医生子女微博关注医生部分说话客观尤专...
20647,10323,1,种毒药断送爱常常听说爱情蜜糖深陷中会沉醉岂知份爱情美会意想原理结束常常意想称爱情毒药爱情会什...
20648,10324,1,果原谅痛苦值降低现贼悔吃麻辣串洗完澡新换衣服全味难受天新闻男熬夜猝死死女抑郁症确诊天昨天加药...


In [36]:
balanced_df_all_cleaned2['tweet_content'] = balanced_df_all_cleaned2['tweet_content'].astype(str)

In [37]:
balanced_df_all_cleaned2['tweet_content'] = balanced_df_all_cleaned2['tweet_content'].apply(lambda x: ' '.join(jieba.cut(x)))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.900 seconds.
Prefix dict has been built successfully.


In [38]:
balanced_df_all_cleaned2

,ID,label,tweet_content
0,15315,0,害真 挺 吃 颜想 岁 足 表达 减肥 决心 姐妹 谈恋爱 什 感觉 全世界 闺蜜 介绍 成...
1,17978,0,拥抱 世界 暖心 动作 生活 中 难免 遇 挫折 坎坷 安 慌乱 许时 拥抱 会 充满 量 ...
2,221,0,雨天 穿会 滑倒 坐水里 亲测 艾特子 赟 相信 家会 意见 麻雀 妈妈 问 麻雀 天扎什 ...
3,16604,0,种 幸福 做 桌菜 然家 朋友 满足 吃 完 没 帮忙 刷碗 老规矩 昨晚 想 做 早餐 奶...
4,20755,0,日 清明 青草 疫情 中 牺牲 医护 员 公安干警 基层干部 线 工作 逝世 胞 表示 沉痛...
...,...,...,...
20645,10321,1,意做件 错事 真 久久 原谅 近 发生 事 情绪 波动 房间 突然 爆哭然 雨天 跑 出 外...
20646,10322,1,太 喜欢 张 教授 说话 特实 危言耸听 然 整体 说 认识 医生 亲戚朋友 医生 医生 子...
20647,10323,1,种 毒药 断送 爱 常常 听说 爱情 蜜糖 深陷 中 会 沉醉 岂知 份 爱情 美 会意 想...
20648,10324,1,果 原谅 痛苦 值 降低 现贼 悔 吃 麻辣 串 洗完 澡 新 换衣服 全味 难受 天 新闻...


In [39]:
# save to csv
balanced_df_all_cleaned2.to_csv('balanced_df_all_cleaned_tokenization.csv', index=False, encoding='utf-8')

In [40]:
# Feature extraction
balanced_df_all_cleaned_tokenization = pd.read_csv('balanced_df_all_cleaned_tokenization.csv')

In [41]:
balanced_df_all_cleaned_tokenization

,ID,label,tweet_content
0,15315,0,害真 挺 吃 颜想 岁 足 表达 减肥 决心 姐妹 谈恋爱 什 感觉 全世界 闺蜜 介绍 成...
1,17978,0,拥抱 世界 暖心 动作 生活 中 难免 遇 挫折 坎坷 安 慌乱 许时 拥抱 会 充满 量 ...
2,221,0,雨天 穿会 滑倒 坐水里 亲测 艾特子 赟 相信 家会 意见 麻雀 妈妈 问 麻雀 天扎什 ...
3,16604,0,种 幸福 做 桌菜 然家 朋友 满足 吃 完 没 帮忙 刷碗 老规矩 昨晚 想 做 早餐 奶...
4,20755,0,日 清明 青草 疫情 中 牺牲 医护 员 公安干警 基层干部 线 工作 逝世 胞 表示 沉痛...
...,...,...,...
20645,10321,1,意做件 错事 真 久久 原谅 近 发生 事 情绪 波动 房间 突然 爆哭然 雨天 跑 出 外...
20646,10322,1,太 喜欢 张 教授 说话 特实 危言耸听 然 整体 说 认识 医生 亲戚朋友 医生 医生 子...
20647,10323,1,种 毒药 断送 爱 常常 听说 爱情 蜜糖 深陷 中 会 沉醉 岂知 份 爱情 美 会意 想...
20648,10324,1,果 原谅 痛苦 值 降低 现贼 悔 吃 麻辣 串 洗完 澡 新 换衣服 全味 难受 天 新闻...


In [42]:
balanced_df_all_cleaned_tokenization2 = balanced_df_all_cleaned_tokenization.copy()

In [43]:
# replace NaN values with an empty string
balanced_df_all_cleaned_tokenization2['tweet_content'] = balanced_df_all_cleaned_tokenization2['tweet_content'].fillna('')

In [44]:
# create a TfidfVectorizer object 将tweet_content列作为特征向量化
vectorizer = TfidfVectorizer()

In [45]:
vectorizer

TfidfVectorizer()

In [46]:
# fit and transform the data
X = vectorizer.fit_transform(balanced_df_all_cleaned_tokenization2['tweet_content'])

In [47]:
X

<20650x983994 sparse matrix of type '<class 'numpy.float64'>'
	with 11403990 stored elements in Compressed Sparse Row format>

In [48]:

y = balanced_df_all_cleaned_tokenization2['label']

In [49]:
X.shape, y.shape

((20650, 983994), (20650,))

In [50]:
#Model RF
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [51]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.4)

In [64]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, y, test_size=0.2)

In [ ]:
#how to identify 13

In [52]:
rfc = RandomForestClassifier(n_estimators=13) # 修改树的棵树， 使用10折交叉验证来确定n_estimators的最佳值为13
rfc.fit(X_train, y_train)

RandomForestClassifier(n_estimators=13)

In [53]:
rfc = RandomForestClassifier(n_estimators=13) # 修改树的棵树， 使用10折交叉验证来确定n_estimators的最佳值为13
rfc.fit(X_train2, y_train2)

RandomForestClassifier(n_estimators=13)

In [65]:
rfc = RandomForestClassifier(n_estimators=13) # 修改树的棵树， 使用10折交叉验证来确定n_estimators的最佳值为13
rfc.fit(X_train3, y_train3)

RandomForestClassifier(n_estimators=13)

In [59]:
y_pred_rfc = rfc.predict(X_test)

In [58]:
y_pred_rfc2 = rfc.predict(X_test2)

In [66]:
y_pred_rfc3 = rfc.predict(X_test3)

In [60]:
# 计算模型评估指标
accuracy_rf = accuracy_score(y_test, y_pred_rfc)
precision_rf = precision_score(y_test, y_pred_rfc)
recall_rf = recall_score(y_test, y_pred_rfc)
f1_rf = f1_score(y_test, y_pred_rfc)

print('Accuracy:', accuracy_rf)
print('Precision:', precision_rf)
print('Recall:', recall_rf)
print('F1 Score:', f1_rf)

Accuracy: 0.9601291364003228
Precision: 0.9652986319652986
Recall: 0.9528985507246377
F1 Score: 0.9590585115199736


In [63]:
# 计算模型评估指标 2
accuracy_rf = accuracy_score(y_test2, y_pred_rfc2)
precision_rf = precision_score(y_test2, y_pred_rfc2)
recall_rf = recall_score(y_test2, y_pred_rfc2)
f1_rf = f1_score(y_test2, y_pred_rfc2)

print('Accuracy:', accuracy_rf)
print('Precision:', precision_rf)
print('Recall:', recall_rf)
print('F1 Score:', f1_rf)

Accuracy: 0.9026634382566586
Precision: 0.9139758442198669
Recall: 0.8907038193610377
F1 Score: 0.9021897810218978


In [67]:
# 计算模型评估指标 2
accuracy_rf = accuracy_score(y_test3, y_pred_rfc3)
precision_rf = precision_score(y_test3, y_pred_rfc3)
recall_rf = recall_score(y_test3, y_pred_rfc3)
f1_rf = f1_score(y_test3, y_pred_rfc3)

print('Accuracy:', accuracy_rf)
print('Precision:', precision_rf)
print('Recall:', recall_rf)
print('F1 Score:', f1_rf)

Accuracy: 0.9096852300242131
Precision: 0.9207873259721555
Recall: 0.9021636876763875
F1 Score: 0.9113803753860775


In [68]:
# normal content
new_tweet_content = ['这是 一条 新的 正常 推文']

# depressed content
# new_tweet_content = ['思 诺思 舒乐安定 代开 疫情 期间 开药 困难 开药 断药 关注 南京 兼职 超话志道 合盆友 解释 快 超话 传送门 南京 兼职']

# feature extraction
new_X = vectorizer.transform(new_tweet_content)

# RF
new_y_pred_rfc = rfc.predict(new_X)

print('\nnew_tweet_content label:')
print('RF:', new_y_pred_rfc)


new_tweet_content label:
RF: [0]


In [74]:

# depressed content
new_tweet_content = ['思 诺思 舒乐安定 代开 疫情 期间 开药 困难 开药 断药 关注 南京 兼职 超话志道 合盆友 解释 快 超话 传送门 南京 兼职']

# feature extraction
new_X = vectorizer.transform(new_tweet_content)

# RF
new_y_pred_rfc = rfc.predict(new_X)

print('\nnew_tweet_content label:')
print('RF:', new_y_pred_rfc)


new_tweet_content label:
RF: [1]


In [71]:
#SVM
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

SVC(kernel='linear')

In [72]:
# 预测测试集的label
y_pred_svc = svc.predict(X_test)

In [73]:
accuracy_svc = accuracy_score(y_test, y_pred_svc)
precision_svc = precision_score(y_test, y_pred_svc)
recall_svc = recall_score(y_test, y_pred_svc)
f1_svc = f1_score(y_test, y_pred_svc)

print('\nSupport Vector Machine:')
print('Accuracy:', accuracy_svc)
print('Precision:', precision_svc)
print('Recall:', recall_svc)
print('F1 Score:', f1_svc)


Support Vector Machine:
Accuracy: 0.9465698143664245
Precision: 0.982174688057041
Recall: 0.9074440052700923
F1 Score: 0.9433316212977231


In [75]:
# normal content
# new_tweet_content = ['这是 一条 新的 正常 推文']

# depressed content
new_tweet_content = ['思 诺思 舒乐安定 代开 疫情 期间 开药 困难 开药 断药 关注 南京 兼职 超话志道 合盆友 解释 快 超话 传送门 南京 兼职']

# feature extraction
new_X = vectorizer.transform(new_tweet_content)

# SVM
new_y_pred_svc = svc.predict(new_X)

print('\nnew_tweet_content label:')
print('SVM:', new_y_pred_svc)


new_tweet_content label:
SVM: [1]


In [76]:
# normal content
new_tweet_content = ['这是 一条 新的 正常 推文']

# depressed content
# new_tweet_content = ['思 诺思 舒乐安定 代开 疫情 期间 开药 困难 开药 断药 关注 南京 兼职 超话志道 合盆友 解释 快 超话 传送门 南京 兼职']

# feature extraction
new_X = vectorizer.transform(new_tweet_content)

# SVM
new_y_pred_svc = svc.predict(new_X)

print('\nnew_tweet_content label:')
print('SVM:', new_y_pred_svc)


new_tweet_content label:
SVM: [0]


In [ ]:
#TextCNN

In [6]:
# Load the dataset
import pandas as pd
balanced_df_all_cleaned_tokenization = pd.read_csv('balanced_df_all_cleaned_tokenization.csv')

In [7]:
balanced_df_all_cleaned_tokenization['tweet_content'] = balanced_df_all_cleaned_tokenization['tweet_content'].fillna('')

In [8]:
df3=balanced_df_all_cleaned_tokenization.copy()

In [5]:
df3.head()

,ID,label,tweet_content
0,15315,0,害真 挺 吃 颜想 岁 足 表达 减肥 决心 姐妹 谈恋爱 什 感觉 全世界 闺蜜 介绍 成...
1,17978,0,拥抱 世界 暖心 动作 生活 中 难免 遇 挫折 坎坷 安 慌乱 许时 拥抱 会 充满 量 ...
2,221,0,雨天 穿会 滑倒 坐水里 亲测 艾特子 赟 相信 家会 意见 麻雀 妈妈 问 麻雀 天扎什 ...
3,16604,0,种 幸福 做 桌菜 然家 朋友 满足 吃 完 没 帮忙 刷碗 老规矩 昨晚 想 做 早餐 奶...
4,20755,0,日 清明 青草 疫情 中 牺牲 医护 员 公安干警 基层干部 线 工作 逝世 胞 表示 沉痛...


In [9]:
# Preprocess the data
X = df3['tweet_content']
y = df3['label']
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [10]:
# Tokenize the sequences using Jieba tokenizer
import jieba 
import jieba.analyse
def tokenize(text):
    return list(jieba.cut(text))
X_train_sequences = [' '.join(tokenize(x)) for x in X_train]
X_test_sequences = [' '.join(tokenize(x)) for x in X_test]


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.717 seconds.
Prefix dict has been built successfully.


In [12]:
import tensorflow as tf

In [13]:
# Convert tokens to integer sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X_train_sequences)

# Convert tokenized sequences to integer sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train_sequences)
X_test_sequences = tokenizer.texts_to_sequences(X_test_sequences)

In [11]:
# Pad sequences
max_sequence_length = 7000  # Set the maximum sequence length for padding
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')


In [12]:
print(X_test_padded.shape)

(6195, 7000)


In [13]:
# Define the model architecture
embedding_dim = 100
num_filters = 100
filter_sizes = [3, 4, 5]

inputs = tf.keras.Input(shape=(max_sequence_length,))
embedding_layer = tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim)(inputs)
conv_layers = []
for filter_size in filter_sizes:
    conv_layer = tf.keras.layers.Conv1D(num_filters, filter_size, activation='relu')(embedding_layer)
    pool_layer = tf.keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers.append(pool_layer)
concatenated = tf.keras.layers.Concatenate()(conv_layers)
dropout = tf.keras.layers.Dropout(0.5)(concatenated)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
history = model.fit(X_train_padded, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_padded, y_test))

# Evaluate the model
y_pred = model.predict(X_test_padded)
y_pred_binary = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)

Epoch 1/10
452/452 [==============================] - 584s 1s/step - loss: 0.2209 - accuracy: 0.9126 - val_loss: 0.0996 - val_accuracy: 0.9656
Epoch 2/10
452/452 [==============================] - 595s 1s/step - loss: 0.0802 - accuracy: 0.9736 - val_loss: 0.0914 - val_accuracy: 0.9671
Epoch 3/10
452/452 [==============================] - 603s 1s/step - loss: 0.0295 - accuracy: 0.9907 - val_loss: 0.0999 - val_accuracy: 0.9663
Epoch 4/10
452/452 [==============================] - 595s 1s/step - loss: 0.0089 - accuracy: 0.9981 - val_loss: 0.1095 - val_accuracy: 0.9653
Epoch 5/10
452/452 [==============================] - 588s 1s/step - loss: 0.0042 - accuracy: 0.9990 - val_loss: 0.1151 - val_accuracy: 0.9664
Epoch 6/10
452/452 [==============================] - 493s 1s/step - loss: 0.0026 - accuracy: 0.9994 - val_loss: 0.1261 - val_accuracy: 0.9653
Epoch 7/10
452/452 [==============================] - 511s 1s/step - loss: 9.3956e-04 - accuracy: 0.9999 - val_loss: 0.1311 - val_accuracy: 0.

In [1]:
# Import necessary libraries
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch


In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm


In [3]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
# Tokenize using Jieba (you have already done this)
X_train_sequences = [' '.join(tokenize(x)) for x in X_train]
X_test_sequences = [' '.join(tokenize(x)) for x in X_test]

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)  # Use a Chinese BERT model

# Convert tokenized sequences to integer sequences
X_train_sequences = tokenizer(X_train_sequences, padding=True, truncation=True, return_tensors="pt")
X_test_sequences = tokenizer(X_test_sequences, padding=True, truncation=True, return_tensors="pt")

# Convert to PyTorch tensors
X_train_tensors = X_train_sequences['input_ids']
X_test_tensors = X_test_sequences['input_ids']
y_train_tensors = torch.tensor(y_train.values)
y_test_tensors = torch.tensor(y_test.values)

In [15]:

# Create TensorDatasets
train_data = TensorDataset(X_train_tensors, y_train_tensors)
test_data = TensorDataset(X_test_tensors, y_test_tensors)

# Create DataLoader
batch_size = 32
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)



In [ ]:
# model.eval()

# y_true = []
# y_pred = []

# with torch.no_grad():
#     for batch in tqdm(test_dataloader, desc="Evaluating"):
#         input_ids, attention_masks, labels = batch
#         outputs = model(input_ids, attention_mask=attention_masks)
#         logits = outputs.logits
#         predictions = torch.argmax(logits, dim=1)
        
#         y_true.extend(labels.cpu().numpy())
#         y_pred.extend(predictions.cpu().numpy())

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids, labels = batch  # Remove attention_masks from unpacking
        outputs = model(input_ids)
        logits = outputs.logits

        # Your evaluation logic here



Evaluating:   3%|█▌                                                            | 5/194 [4:46:50<168:07:51, 3202.49s/it]

In [ ]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
